In [1]:
import pandas as pd
import numpy as np
import optuna
from catboost import CatBoostRegressor
from sklearn.metrics import mean_squared_error, r2_score
from sklearn.model_selection import train_test_split
from datetime import datetime
from sklearn.preprocessing import StandardScaler

In [7]:
train = pd.read_csv('./train22.csv')
val = pd.read_csv('./val.csv')

In [8]:
np.random.seed(10)

In [9]:
train.drop(columns = ['date', 'pickup_date'], inplace = True)
val.drop(columns = ['date', 'pickup_date'], inplace = True)

In [10]:
NUMERIC_FEATURES = ['valid_miles', 'weight']
sc = StandardScaler()
sc.fit(train[NUMERIC_FEATURES])
train[NUMERIC_FEATURES]= sc.transform(train[NUMERIC_FEATURES])
val[NUMERIC_FEATURES]= sc.transform(val[NUMERIC_FEATURES])

In [11]:
X = train.drop('rate', axis=1)
y = train['rate']

X_val = val.drop('rate', axis=1)
y_val = val['rate']

In [12]:
X_train, X_test, y_train, y_test = train_test_split(X, y, test_size=0.2, random_state=42)

In [17]:
categorical_features = ['origin_kma', 'destination_kma', 'transport_type']

In [18]:
def loss(real_rates, predicted_rates):
    return np.average(abs(predicted_rates / real_rates - 1.0)) * 100.0

In [19]:
def objective(trial):
    params = {
        'iterations': trial.suggest_int('iterations', 100, 1000),
        'depth': trial.suggest_int('depth', 3, 15),
        'learning_rate': trial.suggest_float('learning_rate', 0.001, 0.5, log=True),
        'l2_leaf_reg': trial.suggest_float('l2_leaf_reg', 1e-5, 1e1, log=True),
        # Regularization and Complexity Control Parameters
        'bagging_temperature': trial.suggest_float('bagging_temperature', 0.0, 1.0),
        'rsm': trial.suggest_float('rsm', 0.1, 1.0),
        
        # Feature Subsampling
        'subsample': trial.suggest_float('subsample', 0.5, 1.0),
    }
    
    
    catboost = CatBoostRegressor(**params, cat_features = categorical_features)
    catboost.fit(X_train, y_train)
    y_pred = catboost.predict(X_test)
    
    loss_val = loss(y_test, y_pred)
    return loss_val

In [21]:
study = optuna.create_study(direction='minimize')
study.optimize(objective, n_trials=50)

[I 2023-11-06 11:45:04,289] A new study created in memory with name: no-name-cd1ba414-25f8-4018-be21-947a42474f4a


0:	learn: 1.8490334	total: 31.2ms	remaining: 20.6s
1:	learn: 1.8458937	total: 70.6ms	remaining: 23.3s
2:	learn: 1.8426677	total: 106ms	remaining: 23.3s
3:	learn: 1.8392578	total: 142ms	remaining: 23.3s
4:	learn: 1.8359285	total: 179ms	remaining: 23.4s
5:	learn: 1.8326029	total: 216ms	remaining: 23.6s
6:	learn: 1.8295805	total: 256ms	remaining: 23.9s
7:	learn: 1.8264534	total: 294ms	remaining: 24s
8:	learn: 1.8232554	total: 328ms	remaining: 23.7s
9:	learn: 1.8200104	total: 358ms	remaining: 23.3s
10:	learn: 1.8171608	total: 393ms	remaining: 23.2s
11:	learn: 1.8141427	total: 426ms	remaining: 23.1s
12:	learn: 1.8108817	total: 467ms	remaining: 23.3s
13:	learn: 1.8076795	total: 506ms	remaining: 23.4s
14:	learn: 1.8044238	total: 540ms	remaining: 23.2s
15:	learn: 1.8012112	total: 574ms	remaining: 23.1s
16:	learn: 1.7980286	total: 610ms	remaining: 23.1s
17:	learn: 1.7951291	total: 649ms	remaining: 23.2s
18:	learn: 1.7922474	total: 686ms	remaining: 23.2s
19:	learn: 1.7893484	total: 722ms	remaini

[I 2023-11-06 11:45:27,925] Trial 0 finished with value: 18.60550613429892 and parameters: {'iterations': 661, 'depth': 3, 'learning_rate': 0.0030672154818327893, 'l2_leaf_reg': 1.1662972405479544e-05, 'bagging_temperature': 0.3710295556967742, 'rsm': 0.6364214005857557, 'subsample': 0.7261001187124564}. Best is trial 0 with value: 18.60550613429892.


0:	learn: 1.6967063	total: 67.2ms	remaining: 35.8s
1:	learn: 1.5723608	total: 144ms	remaining: 38.3s
2:	learn: 1.4749620	total: 214ms	remaining: 38s
3:	learn: 1.3797541	total: 294ms	remaining: 38.9s
4:	learn: 1.2979161	total: 362ms	remaining: 38.2s
5:	learn: 1.2307333	total: 442ms	remaining: 38.9s
6:	learn: 1.1776434	total: 514ms	remaining: 38.7s
7:	learn: 1.1305202	total: 586ms	remaining: 38.5s
8:	learn: 1.0928171	total: 662ms	remaining: 38.6s
9:	learn: 1.0643305	total: 732ms	remaining: 38.4s
10:	learn: 1.0406000	total: 809ms	remaining: 38.5s
11:	learn: 1.0225926	total: 878ms	remaining: 38.2s
12:	learn: 1.0067049	total: 956ms	remaining: 38.3s
13:	learn: 0.9798495	total: 1.03s	remaining: 38.3s
14:	learn: 0.9604681	total: 1.1s	remaining: 38.1s
15:	learn: 0.9445814	total: 1.17s	remaining: 38s
16:	learn: 0.9331284	total: 1.24s	remaining: 37.8s
17:	learn: 0.9209535	total: 1.31s	remaining: 37.6s
18:	learn: 0.9123375	total: 1.37s	remaining: 37.2s
19:	learn: 0.9039872	total: 1.45s	remaining: 

[I 2023-11-06 11:46:08,222] Trial 1 finished with value: 10.710351963190638 and parameters: {'iterations': 534, 'depth': 6, 'learning_rate': 0.14536638178204714, 'l2_leaf_reg': 0.00019833627034022215, 'bagging_temperature': 0.29472778115515075, 'rsm': 0.736505743099913, 'subsample': 0.7107796947664411}. Best is trial 1 with value: 10.710351963190638.


...

[I 2023-11-06 15:42:27,705] Trial 47 finished with value: 10.013167546899682 and parameters: {'iterations': 870, 'depth': 10, 'learning_rate': 0.06549498757577057, 'l2_leaf_reg': 0.038473359255958274, 'bagging_temperature': 0.5760324083942994, 'rsm': 0.527778860394151, 'subsample': 0.9297450516091078}. Best is trial 46 with value: 9.759468343832033.


0:	learn: 1.8093905	total: 125ms	remaining: 1m 37s
1:	learn: 1.7628252	total: 286ms	remaining: 1m 51s
2:	learn: 1.7184562	total: 465ms	remaining: 2m
3:	learn: 1.6764751	total: 572ms	remaining: 1m 51s
4:	learn: 1.6346222	total: 735ms	remaining: 1m 53s
5:	learn: 1.5948929	total: 889ms	remaining: 1m 54s
6:	learn: 1.5571186	total: 1.07s	remaining: 1m 57s
7:	learn: 1.5219949	total: 1.24s	remaining: 1m 59s
8:	learn: 1.4884904	total: 1.4s	remaining: 1m 59s
9:	learn: 1.4573900	total: 1.57s	remaining: 2m 1s
10:	learn: 1.4267937	total: 1.74s	remaining: 2m 1s
11:	learn: 1.3978329	total: 1.9s	remaining: 2m 1s
12:	learn: 1.3700944	total: 2.07s	remaining: 2m 2s
13:	learn: 1.3444392	total: 2.23s	remaining: 2m 2s
14:	learn: 1.3197254	total: 2.4s	remaining: 2m 2s
15:	learn: 1.2969588	total: 2.56s	remaining: 2m 2s
16:	learn: 1.2745178	total: 2.73s	remaining: 2m 2s
17:	learn: 1.2745178	total: 2.74s	remaining: 1m 56s
18:	learn: 1.2529652	total: 2.91s	remaining: 1m 56s
19:	learn: 1.2276985	total: 3.08s	rem

[I 2023-11-06 15:44:39,080] Trial 48 finished with value: 10.156143530232463 and parameters: {'iterations': 780, 'depth': 11, 'learning_rate': 0.03878398518263289, 'l2_leaf_reg': 0.023071047975972585, 'bagging_temperature': 0.5188429464257971, 'rsm': 0.5852183089565893, 'subsample': 0.9351613494177266}. Best is trial 46 with value: 9.759468343832033.


0:	learn: 1.7962649	total: 104ms	remaining: 1m 36s
1:	learn: 1.7440770	total: 249ms	remaining: 1m 55s
2:	learn: 1.6861860	total: 628ms	remaining: 3m 14s
3:	learn: 1.6327107	total: 749ms	remaining: 2m 53s
4:	learn: 1.5790549	total: 1.08s	remaining: 3m 21s
5:	learn: 1.5296518	total: 1.46s	remaining: 3m 45s
6:	learn: 1.4845140	total: 1.84s	remaining: 4m 2s
7:	learn: 1.4411207	total: 2.17s	remaining: 4m 11s
8:	learn: 1.4013784	total: 2.48s	remaining: 4m 14s
9:	learn: 1.3639907	total: 2.87s	remaining: 4m 24s
10:	learn: 1.3282369	total: 3.24s	remaining: 4m 31s
11:	learn: 1.2958823	total: 3.6s	remaining: 4m 36s
12:	learn: 1.2658619	total: 3.98s	remaining: 4m 41s
13:	learn: 1.2369990	total: 4.38s	remaining: 4m 47s
14:	learn: 1.2088198	total: 4.78s	remaining: 4m 52s
15:	learn: 1.1841668	total: 5.14s	remaining: 4m 54s
16:	learn: 1.1613080	total: 5.46s	remaining: 4m 53s
17:	learn: 1.1374084	total: 5.84s	remaining: 4m 56s
18:	learn: 1.1176136	total: 6.24s	remaining: 4m 59s
19:	learn: 1.0978304	tot

[I 2023-11-06 15:51:25,903] Trial 49 finished with value: 9.815176996542895 and parameters: {'iterations': 932, 'depth': 13, 'learning_rate': 0.05122460363085036, 'l2_leaf_reg': 0.13180850497657873, 'bagging_temperature': 0.446981620646622, 'rsm': 0.6663539071385491, 'subsample': 0.972882665978289}. Best is trial 46 with value: 9.759468343832033.


In [22]:
best_params = study.best_params
best_params

{'iterations': 823,
 'depth': 14,
 'learning_rate': 0.07617184138306553,
 'l2_leaf_reg': 0.06964226095149612,
 'bagging_temperature': 0.5999648949295984,
 'rsm': 0.6685873488544701,
 'subsample': 0.9255643423817381}

In [23]:
catboost = CatBoostRegressor(**best_params, cat_features = categorical_features)
catboost.fit(X_train, y_train)

0:	learn: 1.7695904	total: 108ms	remaining: 1m 28s
1:	learn: 1.6952804	total: 250ms	remaining: 1m 42s
2:	learn: 1.6130126	total: 887ms	remaining: 4m 2s
3:	learn: 1.5401844	total: 1.36s	remaining: 4m 37s
4:	learn: 1.4703322	total: 2.02s	remaining: 5m 29s
5:	learn: 1.4074183	total: 2.59s	remaining: 5m 53s
6:	learn: 1.3501248	total: 3.23s	remaining: 6m 16s
7:	learn: 1.2989587	total: 3.77s	remaining: 6m 23s
8:	learn: 1.2531032	total: 4.46s	remaining: 6m 42s
9:	learn: 1.2122862	total: 5.05s	remaining: 6m 50s
10:	learn: 1.1726602	total: 5.66s	remaining: 6m 57s
11:	learn: 1.1416586	total: 5.83s	remaining: 6m 34s
12:	learn: 1.1097217	total: 6.49s	remaining: 6m 44s
13:	learn: 1.0796269	total: 7.02s	remaining: 6m 45s
14:	learn: 1.0548643	total: 7.51s	remaining: 6m 44s
15:	learn: 1.0288938	total: 8.11s	remaining: 6m 49s
16:	learn: 1.0064172	total: 8.77s	remaining: 6m 55s
17:	learn: 0.9855174	total: 9.36s	remaining: 6m 58s
18:	learn: 0.9652167	total: 9.85s	remaining: 6m 56s
19:	learn: 0.9455780	to

In [24]:
loss(y_val, catboost.predict(X_val))

12.453595152751543

In [29]:
# Посмотрим на важность фичей:
pd.DataFrame({
    'feature_importance': catboost.feature_importances_,
    'feature_names': catboost.feature_names_
})

,feature_importance,feature_names
0,29.448362,valid_miles
1,5.320651,transport_type
2,12.816098,weight
3,27.105953,origin_kma
4,25.308936,destination_kma


In [34]:
test = pd.read_csv('./test.csv', index_col=0)
test.drop(columns = ['date', 'pickup_date'], inplace = True)

In [35]:
test[NUMERIC_FEATURES]= sc.transform(test[NUMERIC_FEATURES])

In [36]:
test['catboost_predict'] = catboost.predict(test)

In [39]:
# Сохраняем результат, оставляя колонку с индексом (это необходимо для дальнейшей сверки)
test.to_csv('test_predict_catboost.csv')